<a href="https://colab.research.google.com/github/Ayanlola2002/DATA-SCIENCE-PROJECTS/blob/master/NeedEnergyAPI_Data_Wrangling2withJoe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import pandas as pd
import numpy as np
from datetime import date, datetime
import holidays
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None

In [57]:
'''df_5min=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_five_min_id_47803.csv')
df_hourly=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_hourly_id_47803.csv')
df_daily=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_daily_id_47803.csv')
df_monthly=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_monthly_id_47803.csv')
df_public_holiday=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/public_holidays_weekends_ZIM.csv')'''


"df_5min=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_five_min_id_47803.csv')\ndf_hourly=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_hourly_id_47803.csv')\ndf_daily=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_daily_id_47803.csv')\ndf_monthly=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_monthly_id_47803.csv')\ndf_public_holiday=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/public_holidays_weekends_ZIM.csv')"

In [58]:
##Wrangling Helper Functions to Use
"""
This function do the reading of data file from file path(i.e drive)
"""
def do_read_data(datafile, dateCol):
    ''' Read the excel file'''
    df = pd.read_excel(datafile, index_col= dateCol, parse_dates=True)
    return df

"""
This function Shows Information of the Data file
Like The Header columns, Dimension of Data
"""

def do_data_information(datafile, info = False, head = True, shape = True):
    ''' Look at the info of the data'''
    
    if not isinstance(datafile, pd.DataFrame):
        df = read_data(datafile)
    else:
        df = datafile
    
    ## Information of data
    if info:
        df.info()
        
    print('--------------------------------------------------------------------')
    print('First rows')
    # Show first rows
    if head:
        print(df.head())
        
    print('--------------------------------------------------------------------')
    print('Dimension of the data')
    ## Dimension of data
    if shape:
        print(f'This data has {df.shape[0]} rows and {df.shape[1]} variables') 
        
###This function returns the number of distinct values from any columns       
def do_unique_values(df, col):
    ''' Find the unique values of a columns'''
    return df[col].nunique()


###This function statistically describe the data values
def do_description(df, col = None):
    ''' Give the statistic summary'''
    
    ## Summary statistic
    if col is not None:
        stat = df[col].describe()
    else:
        stat = df.describe()

    ## Change the name of the 50% index to median
    idx = stat.index.tolist()
    idx[5] = 'median'
    stat.index = idx
    return stat

###This function finds % Missing values
def check_missing_value(data):
    ## percentage of missing values
    n = data.isnull().sum().sort_values(ascending=False)/ len(data) * 100
    return n

def check_missing_values_table(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
  '% of Total Values', ascending=False).round(1)
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
      "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  return mis_val_table_ren_columns


###This function finds the amount of each values in a dataset
def do_valueCount(data, col, perc = False):
    ''' calculate the proportion in fraction or percentage'''
    if not perc:
      return data[col].value_counts()
    else:
      return data[col].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'    
### This function count number of unique values for all columns in a dataset
def do_nunique_value(df):
  print('The Numver of unique values per columns are as follows')
  print(df.agg(['nunique']).T)


###For date time conversion start here   
def do_date_col_convertion(df, column_name):
  df[column_name]=pd.to_datetime(df[column_name])
  return df[column_name]

def do_time_col_convertion(df, column_name):
    return pd.to_timedelta(df[column_name])


def do_datepart_extraction(data):
  data['date'] = pd.to_datetime(data['date'])
  data['dateOnly']=data['date'].dt.date
  data['year'] = data['date'].dt.year
  data['quarter']=data['date'].dt.quarter
  data['month'] = data['date'].dt.month
  data['weekday'] = data['date'].dt.weekday
  data['day_name'] = data['date'].dt.day_name()
  data['dayofyear'] = data['date'].dt.dayofyear
  data['day'] =data['date'].dt.day
  data['hour'] = data['date'].dt.hour
  data['minute'] =data['date'].dt.minute
  #data['Season']=data['month'].apply(do_year_season)
  data['season'] = data.date.map(do_season_of_date)
  data['day_section']=data['hour'].apply(do_day_sections)
  return data
   

def do_season_of_date(date):
  year = str(date.year)
  seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
              'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
              'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
  if date in seasons['spring']:
    return 'spring'
  if date in seasons['summer']:
    return 'summer'
  if date in seasons['autumn']:
    return 'autumn'
  else:
    return 'winter'

# Assuming df has a date column of type `datetime`
#df['season'] = df.date.map(season_of_date)


def do_day_sections(x):
  if (x > 4) and (x <= 8):
    return 'Early Morning'
  elif (x > 8) and (x <= 12 ):
    return 'Morning'
  elif (x > 12) and (x <= 16):
    return'Noon'
  elif (x > 16) and (x <= 20) :
    return 'Evening'
  elif (x > 20) and (x <= 24):
    return'Night'
  elif (x <= 4):
    return'Late Night'




def get_holidays(startYear = 2018, endYear = 2025, countryCode = 'ZA'):
  """
    Takes in a start and end date, and start and end year.
    
    Produces a dataframe with a daily date and columns:
    holiday - 'Y' for holiday
    holidayName - name of the holiday if holiday is 'Y'
    
    Returns a dataframe
  """
  holidayDict = {}
  for i in range(startYear, endYear):
    for date, name in sorted(holidays.CountryHoliday(countryCode,years=[i]).items()):
      holidayDict[date] = name
      holiday_df = pd.DataFrame(list(holidayDict.items()),columns = ['day','holidayName'])
      holiday_df['day'] = pd.to_datetime(holiday_df['day'], format='%Y-%M-%d').dt.date
    return holiday_df
    

    
def get_days(start = '1/1/2018',startYear = 2018, end = '31/12/2025',endYear = 2025, countryCode = 'ZA'):
    """
    Takes in a start and end date, and start and end year.
    
    Produces a dataframe with a daily date and columns:
    weekend - 'Y' for weendend and 'N' for workday
    dayOfweek - numerical day of the week identifier 0 for monday
    weekNum - numerical number of the week 
    holiday - 'Y' for holiday
    holidayName - name of the holiday if holiday is 'Y'
    
    Returns a dataframe
    """
    
    #generate the range of daily dates
    dates = pd.date_range(start = start, end = end)
    date_df = pd.DataFrame(dates, columns = ['day'])
    date_df['day'] = pd.to_datetime(date_df['day'], format='%Y-%M-%d')
    country_holidays = get_holidays(startYear = startYear, endYear = endYear, countryCode = countryCode)
    
    date_df['dayName'] = pd.DatetimeIndex(date_df['day']).day_name()
    date_df['dayOfWeek'] = date_df['day'].dt.dayofweek
    date_df['weekend'] = date_df['dayOfWeek'].apply(lambda x: 'Y' if x>4 else 'N')
    date_df['weekNum'] = date_df['day'].dt.week
    date_df['holiday'] = date_df['day'].apply(lambda x: 'Y' if x in country_holidays['day'].values else 'N')
    date_df['day'] = date_df['day'].dt.date
    date_df = date_df.merge(country_holidays, on='day', how='left', indicator=False)
    
    date_df.to_csv(f'../data/public_holidays_weekends.csv', index=False)
    return date_df

###Functions to impute nan values for numerical varriable
def Impute_nan(df):
  numericalcol = [f for f in df_5min.columns if df_5min.dtypes[f] != 'object']
  missing_col=df_5min[numericalcol].columns[df_5min[numericalcol].isnull().any()].tolist()
  #Technique 1: Using mean to impute the missing values
  for i in missing_col:
    df.loc[df.loc[:,i].isnull(),i]=df.loc[:,i].mean()

def Impute_na(df):
  numericalcol = [f for f in df_5min.columns if df_5min.dtypes[f] != 'object']
  missing_col=df_5min[numericalcol].columns[df_5min[numericalcol].isnull().any()].tolist()
  #Technique 1: Using mean to impute the missing values
  for i in missing_col:
     df[i].fillna(mean,inplace=True)


#dataList=[df_5min,df_hourly,df_daily,df_monthly]

def do_wrangling_task(dL):
  for i in range(len(dL)):
    print(do_data_information(dL[i]))
    print('===============================================')
    print(check_missing_values_table(dL[i]))
    print('===============================================')
    print(do_nunique_value(dL[i]))
    print('===============================================')
    #Feature Engineering
    dL[i]=do_datepart_extraction(dL[i])
    print('===============================================')
    dL[i].date.min(),dL[i].date.max()
    print('===============================================')
    dL[i].consumption.min(),dL[i].consumption.max()
    print('===============================================')
    #checking for days where there weere no consumptions
    dL[i][dL[i].consumption==0].value_counts().count(),dL[i][dL[i].consumption==0].value_counts()
    print('===============================================')
    dL[i]['Public_Holiday'] = dL[i]['dateOnly'].map(mapping)
    print('===============================================')
    dL[i]['Public_Holiday']=dL[i]['Public_Holiday'].apply(lambda x:np.NaN if isinstance(x, str) and (x.isspace() or not x) else x).replace(np.NaN,'Non_Holiday')
    print('===============================================')
    dL[i]=dL[i][['date','timestamp','year','quarter','month','dayofyear','day','day_name','hour','minute','season','day_section','Public_Holiday','consumption','solar','alwaysOn','gridImport','gridExport','selfConsumption','selfSufficiency','active','reactive','voltages','phaseVoltages','currentHarmonics','voltageHarmonics']]
    print('===============================================')
    print('===============================================')

# load helper functions for dynamic import
def import_package(name):
    try:
        module = __import__(name)
    except:
        if name == 'pandas_profiling':
          ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
        module = __import__(name)
    try:
        print(f'{name}'+ ' Version:', module.__version__)
    except AttributeError:
        pass
    return module


# JMcQ
def plot_period(df, date1, date2, column, title):
    
    df = df.loc[date1:date2]
    
    fig, axs = plt.subplots(1,1, figsize = (12, 6))
    
    fig.suptitle(title)
    
    x = df['datetime']
    
    axs.plot(x, df[column], label=column)
    axs.legend(loc="best")

    plt.show()

# JMcQ
def create_monthly_profiles(df, column):
    
    months = df['monthName'].unique()
    
    df = df.groupby([df['weekend'], df['monthName'], df['hour']]).mean()
       
    fig, axs = plt.subplots(1, 2, figsize=(10,6), sharey=True, sharex=True)
    
    fig.suptitle('Weekend and Weekday Profiles')
    
    x = list(range(0,24))
    plt.xticks(x)
    
    # Loop through each month of the year and plot profile
    for month in months:
        
        axs[0].plot(x, df.loc[0, month][column], label=month)
        axs[0].legend(loc="best")
        axs[0].set_title('Weekday')

        axs[1].plot(x, df.loc[1, month][column], label=month)
        axs[1].legend(loc="best")
        axs[1].set_title('Weekend')
        
            
    plt.show()
    
    return df

def load_data(file):
    df = pd.read_csv(file)
    df = df.astype({'date':'datetime64[ns]'})
    df.sort_values(by='date', inplace=True)
    return df

#df['holidays']=df['date'].apply(lambda(x: 'Hols' if x in get_holidays()['day'].values else 'Non-Hols'))



In [59]:
asset = "/content/drive/MyDrive/NEEDENERGY/data_five_min_id_47803.csv"

In [60]:
pd = import_package('pandas')
profile = import_package('pandas_profiling')
np = import_package('numpy')
tz = import_package('dateutil').tz
datetime = import_package('datetime').datetime

pandas Version: 1.1.5
numpy Version: 1.19.5
dateutil Version: 2.8.1


In [61]:
df_5min = load_data(asset)

In [62]:
df_5min.head()

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics
0,2020-03-15 18:15:00,1584288900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -81.6, -60.0, 30.2, -230.6, -0...","[0.5, 0.9, 0.0, 55.4, 132.4, 14.0, 137.5, 0.2,...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1,2020-03-15 18:20:00,1584289200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.8, 0.0, -82.2, -58.5, 30.5, -233.1, -0...","[0.5, 0.9, 0.0, 55.7, 131.9, 14.3, 144.3, 0.2,...","[246.1, None, None]","[246.1, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
2,2020-03-15 18:25:00,1584289500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 28.5, -26.5, -74.4, -79.2, 55.5, -228.5,...","[0.5, 19.0, 8.8, 79.6, 137.6, 29.4, 175.0, 0.4...","[246.4, None, None]","[246.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
3,2020-03-15 18:30:00,1584289800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.5, 21.3, -18.6, -76.9, -54.7, 48.7, -227.8,...","[2.2, 13.3, 6.5, 72.4, 135.9, 24.8, 152.8, 0.8...","[246.0, None, None]","[246.0, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
4,2020-03-15 18:35:00,1584290100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -82.6, -33.2, 30.7, 96.6, 0.2,...","[0.5, 0.9, 0.0, 55.2, 128.1, 13.9, 87.7, 0.7, ...","[246.2, None, None]","[246.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."


Data Quality Information:
df_5min : The data has 95649 rows and 15 features

In [63]:
#basic profiling
df_5min.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95649 entries, 0 to 95648
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              95649 non-null  datetime64[ns]
 1   timestamp         95649 non-null  int64         
 2   consumption       95649 non-null  float64       
 3   solar             95649 non-null  float64       
 4   alwaysOn          95649 non-null  float64       
 5   gridImport        95649 non-null  float64       
 6   gridExport        95649 non-null  float64       
 7   selfConsumption   95649 non-null  float64       
 8   selfSufficiency   95649 non-null  float64       
 9   active            95649 non-null  object        
 10  reactive          95649 non-null  object        
 11  voltages          95649 non-null  object        
 12  phaseVoltages     95649 non-null  object        
 13  currentHarmonics  95649 non-null  object        
 14  voltageHarmonics  9564

In [64]:
#statistcal summary
df_5min.describe


<bound method NDFrame.describe of                      date      timestamp  consumption   solar  alwaysOn  \
0     2020-03-15 18:15:00  1584288900000        0.000   0.000     0.000   
1     2020-03-15 18:20:00  1584289200000        0.000   0.000     0.000   
2     2020-03-15 18:25:00  1584289500000        0.000   0.000     0.000   
3     2020-03-15 18:30:00  1584289800000        0.000   0.000     0.000   
4     2020-03-15 18:35:00  1584290100000        0.000   0.000     0.000   
...                   ...            ...          ...     ...       ...   
95644 2021-02-12 01:35:00  1613086500000      191.070  41.140  1006.212   
95645 2021-02-12 01:40:00  1613086800000      166.362  47.789  1006.212   
95646 2021-02-12 01:45:00  1613087100000      155.303  41.008  1006.212   
95647 2021-02-12 01:50:00  1613087400000      156.402  48.199  1006.212   
95648 2021-02-12 01:55:00  1613087700000      153.934  46.152  1006.212   

       gridImport  gridExport  selfConsumption  selfSufficiency  

In [65]:
#List of features
df_5min.columns

Index(['date', 'timestamp', 'consumption', 'solar', 'alwaysOn', 'gridImport',
       'gridExport', 'selfConsumption', 'selfSufficiency', 'active',
       'reactive', 'voltages', 'phaseVoltages', 'currentHarmonics',
       'voltageHarmonics'],
      dtype='object')

In [66]:
#profile.ProfileReport(df_5min)
!pip install pandas-profiling[notebook]

In [67]:
#profile.ProfileReport(df_5min)

In [68]:
df_5min_new = df_5min.copy()

In [69]:
df_5min_new.head()

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics
0,2020-03-15 18:15:00,1584288900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -81.6, -60.0, 30.2, -230.6, -0...","[0.5, 0.9, 0.0, 55.4, 132.4, 14.0, 137.5, 0.2,...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1,2020-03-15 18:20:00,1584289200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.8, 0.0, -82.2, -58.5, 30.5, -233.1, -0...","[0.5, 0.9, 0.0, 55.7, 131.9, 14.3, 144.3, 0.2,...","[246.1, None, None]","[246.1, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
2,2020-03-15 18:25:00,1584289500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 28.5, -26.5, -74.4, -79.2, 55.5, -228.5,...","[0.5, 19.0, 8.8, 79.6, 137.6, 29.4, 175.0, 0.4...","[246.4, None, None]","[246.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
3,2020-03-15 18:30:00,1584289800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.5, 21.3, -18.6, -76.9, -54.7, 48.7, -227.8,...","[2.2, 13.3, 6.5, 72.4, 135.9, 24.8, 152.8, 0.8...","[246.0, None, None]","[246.0, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
4,2020-03-15 18:35:00,1584290100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -82.6, -33.2, 30.7, 96.6, 0.2,...","[0.5, 0.9, 0.0, 55.2, 128.1, 13.9, 87.7, 0.7, ...","[246.2, None, None]","[246.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."


In [70]:
#Check for duplicates
df_5min_new.index[df_5min_new['date'].duplicated(keep=False)==True]

Int64Index([], dtype='int64')

In [71]:
# Check for missing info using Missingno Package
msno = import_package('missingno')
msno.matrix(df_5min_new)

missingno Version: 0.4.2


In [72]:
# Check if there are any periods of zero values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
features = df_5min_new.select_dtypes(include=numerics).columns 
for feature in features: 
    zero_results = df_5min_new[df_5min_new[feature]==0]
zero_results.head()

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics
0,2020-03-15 18:15:00,1584288900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -81.6, -60.0, 30.2, -230.6, -0...","[0.5, 0.9, 0.0, 55.4, 132.4, 14.0, 137.5, 0.2,...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1,2020-03-15 18:20:00,1584289200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.8, 0.0, -82.2, -58.5, 30.5, -233.1, -0...","[0.5, 0.9, 0.0, 55.7, 131.9, 14.3, 144.3, 0.2,...","[246.1, None, None]","[246.1, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
2,2020-03-15 18:25:00,1584289500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 28.5, -26.5, -74.4, -79.2, 55.5, -228.5,...","[0.5, 19.0, 8.8, 79.6, 137.6, 29.4, 175.0, 0.4...","[246.4, None, None]","[246.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
3,2020-03-15 18:30:00,1584289800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.5, 21.3, -18.6, -76.9, -54.7, 48.7, -227.8,...","[2.2, 13.3, 6.5, 72.4, 135.9, 24.8, 152.8, 0.8...","[246.0, None, None]","[246.0, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
4,2020-03-15 18:35:00,1584290100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -82.6, -33.2, 30.7, 96.6, 0.2,...","[0.5, 0.9, 0.0, 55.2, 128.1, 13.9, 87.7, 0.7, ...","[246.2, None, None]","[246.2, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."


In [73]:
# Check for any missing time steps, create hourly datetime index and compare to length of dataframe
start = df_5min_new['date'].min()
end = df_5min_new['date'].max()
full_index = pd.date_range(start, end, freq='H')
len(full_index) == len(df_5min_new)

False

In [74]:
df_5min_new.set_index('date', inplace = True)
df_5min_new = df_5min_new.reindex(full_index)
df_5min_new.index

DatetimeIndex(['2020-03-15 18:15:00', '2020-03-15 19:15:00',
               '2020-03-15 20:15:00', '2020-03-15 21:15:00',
               '2020-03-15 22:15:00', '2020-03-15 23:15:00',
               '2020-03-16 00:15:00', '2020-03-16 01:15:00',
               '2020-03-16 02:15:00', '2020-03-16 03:15:00',
               ...
               '2021-02-11 16:15:00', '2021-02-11 17:15:00',
               '2021-02-11 18:15:00', '2021-02-11 19:15:00',
               '2021-02-11 20:15:00', '2021-02-11 21:15:00',
               '2021-02-11 22:15:00', '2021-02-11 23:15:00',
               '2021-02-12 00:15:00', '2021-02-12 01:15:00'],
              dtype='datetime64[ns]', length=8000, freq='H')

In [75]:
#Have a look at missing values
print(f"{df_5min_new.isnull().sum()}\n\n")
print(f"Percentage of missing values:\t{(df_5min_new['consumption'].isnull().sum()/df_5min_new['timestamp'].count())*100}")

timestamp           28
consumption         28
solar               28
alwaysOn            28
gridImport          28
gridExport          28
selfConsumption     28
selfSufficiency     28
active              28
reactive            28
voltages            28
phaseVoltages       28
currentHarmonics    28
voltageHarmonics    28
dtype: int64


Percentage of missing values:	0.35122930255895635


Feature Engineering Time Series

In [76]:
df_5min_new.reset_index(inplace=True)
df_5min_new.rename(columns = {'index':'datetime'}, inplace = True)
df_5min_new.head()

,datetime,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics
0,2020-03-15 18:15:00,1.584289e+12,0.000,0.0,0.0,0.000,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -81.6, -60.0, 30.2, -230.6, -0...","[0.5, 0.9, 0.0, 55.4, 132.4, 14.0, 137.5, 0.2,...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1,2020-03-15 19:15:00,1.584292e+12,35.615,0.0,0.0,35.615,0.0,0.0,0.0,"[0.1, 20.3, 0.0, None, 0.0, 47.0, 237.2, 0.3, ...","[0.5, 1.9, 0.0, None, 0.1, 13.6, 155.9, 0.2, N...","[247.3, None, None]","[247.3, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
2,2020-03-15 20:15:00,1.584296e+12,49.976,0.0,0.0,49.976,0.0,0.0,0.0,"[0.1, 3.8, 0.0, None, None, 33.4, 117.8, 0.4, ...","[0.5, 0.9, 0.0, None, None, 14.2, 147.8, 0.3, ...","[246.7, None, None]","[246.7, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
3,2020-03-15 21:15:00,1.584300e+12,67.103,0.0,0.0,67.103,0.0,0.0,0.0,"[0.1, 3.7, 0.0, None, None, 27.4, 92.3, 0.4, N...","[0.5, 0.9, 0.0, None, None, 12.1, 113.0, 0.3, ...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
4,2020-03-15 22:15:00,1.584303e+12,49.070,0.0,0.0,49.070,0.0,0.0,0.0,"[0.1, 3.7, 0.0, None, None, 27.3, 91.4, 0.4, N...","[0.5, 0.9, 0.0, None, None, 12.6, 113.2, 0.3, ...","[243.8, None, None]","[243.8, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."


In [77]:
#Feature Engineering of Time Series
#df_5min_new['datetime']=pd.to_datetime(df_5min_new['date'])
df_5min_new['date'] = df_5min_new['datetime'].dt.strftime('%Y-%m-%d')
df_5min_new['year'] = df_5min_new['datetime'].dt.year 
df_5min_new['dayOfYear'] = df_5min_new['datetime'].dt.dayofyear
df_5min_new['month'] = df_5min_new['datetime'].dt.month 
df_5min_new['monthName'] = pd.DatetimeIndex(df_5min_new['datetime']).month_name()
df_5min_new['week'] = df_5min_new['datetime'].dt.week
df_5min_new['day'] = df_5min_new['datetime'].dt.day
df_5min_new['dayName'] = pd.DatetimeIndex(df_5min_new['datetime']).day_name()
df_5min_new['hour'] = df_5min_new['datetime'].dt.hour
df_5min_new['minute'] = df_5min_new['datetime'].dt.minute
df_5min_new['dayOfWeek'] = df_5min_new['datetime'].dt.dayofweek
df_5min_new['weekend'] = df_5min_new['dayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)
df_5min_new['time'] = df_5min_new['datetime'].dt.time 
df_5min_new['time_Str'] = df_5min_new['datetime'].dt.time.astype(str)   
df_5min_new['dayMonth'] = df_5min_new["datetime"].dt.strftime('%m-%d')
df_5min_new['hourMinute'] = df_5min_new["datetime"].dt.strftime('%H:%M')

bins = [0,4,8,12,16,20,24]
labels = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
df_5min_new['session'] = pd.cut(df_5min_new['hour'], bins=bins, labels=labels, include_lowest=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


Remove Duplicates

In [78]:
df_5min_new = df_5min_new[~df_5min_new['datetime'].duplicated(keep = 'first')]
df_5min_new.head()

,datetime,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics,date,year,dayOfYear,month,monthName,week,day,dayName,hour,minute,dayOfWeek,weekend,time,time_Str,dayMonth,hourMinute,session
0,2020-03-15 18:15:00,1.584289e+12,0.000,0.0,0.0,0.000,0.0,0.0,0.0,"[0.1, 3.7, 0.0, -81.6, -60.0, 30.2, -230.6, -0...","[0.5, 0.9, 0.0, 55.4, 132.4, 14.0, 137.5, 0.2,...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,75,3,March,11,15,Sunday,18,15,6,1,18:15:00,18:15:00,03-15,18:15,Eve
1,2020-03-15 19:15:00,1.584292e+12,35.615,0.0,0.0,35.615,0.0,0.0,0.0,"[0.1, 20.3, 0.0, None, 0.0, 47.0, 237.2, 0.3, ...","[0.5, 1.9, 0.0, None, 0.1, 13.6, 155.9, 0.2, N...","[247.3, None, None]","[247.3, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,75,3,March,11,15,Sunday,19,15,6,1,19:15:00,19:15:00,03-15,19:15,Eve
2,2020-03-15 20:15:00,1.584296e+12,49.976,0.0,0.0,49.976,0.0,0.0,0.0,"[0.1, 3.8, 0.0, None, None, 33.4, 117.8, 0.4, ...","[0.5, 0.9, 0.0, None, None, 14.2, 147.8, 0.3, ...","[246.7, None, None]","[246.7, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,75,3,March,11,15,Sunday,20,15,6,1,20:15:00,20:15:00,03-15,20:15,Eve
3,2020-03-15 21:15:00,1.584300e+12,67.103,0.0,0.0,67.103,0.0,0.0,0.0,"[0.1, 3.7, 0.0, None, None, 27.4, 92.3, 0.4, N...","[0.5, 0.9, 0.0, None, None, 12.1, 113.0, 0.3, ...","[244.5, None, None]","[244.5, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,75,3,March,11,15,Sunday,21,15,6,1,21:15:00,21:15:00,03-15,21:15,Night
4,2020-03-15 22:15:00,1.584303e+12,49.070,0.0,0.0,49.070,0.0,0.0,0.0,"[0.1, 3.7, 0.0, None, None, 27.3, 91.4, 0.4, N...","[0.5, 0.9, 0.0, None, None, 12.6, 113.2, 0.3, ...","[243.8, None, None]","[243.8, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...",2020-03-15,2020,75,3,March,11,15,Sunday,22,15,6,1,22:15:00,22:15:00,03-15,22:15,Night


Missing Data Imputation

In [79]:
#Investigate where missing row/hours are
# Consumption null values occur in continuous period 
# Do not use simple forward fill, impute average day profile for both days
df_5min_new[df_5min_new['consumption'].isnull()]

,datetime,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics,date,year,dayOfYear,month,monthName,week,day,dayName,hour,minute,dayOfWeek,weekend,time,time_Str,dayMonth,hourMinute,session
179,2020-03-23 05:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-23,2020,83,3,March,13,23,Monday,5,15,0,0,05:15:00,05:15:00,03-23,05:15,Early Morning
600,2020-04-09 18:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-09,2020,100,4,April,15,9,Thursday,18,15,3,0,18:15:00,18:15:00,04-09,18:15,Eve
1579,2020-05-20 13:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-20,2020,141,5,May,21,20,Wednesday,13,15,2,0,13:15:00,13:15:00,05-20,13:15,Noon
1632,2020-05-22 18:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-22,2020,143,5,May,21,22,Friday,18,15,4,0,18:15:00,18:15:00,05-22,18:15,Eve
1633,2020-05-22 19:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-22,2020,143,5,May,21,22,Friday,19,15,4,0,19:15:00,19:15:00,05-22,19:15,Eve
1634,2020-05-22 20:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-22,2020,143,5,May,21,22,Friday,20,15,4,0,20:15:00,20:15:00,05-22,20:15,Eve
1635,2020-05-22 21:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-22,2020,143,5,May,21,22,Friday,21,15,4,0,21:15:00,21:15:00,05-22,21:15,Night
1636,2020-05-22 22:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-22,2020,143,5,May,21,22,Friday,22,15,4,0,22:15:00,22:15:00,05-22,22:15,Night
1637,2020-05-22 23:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-22,2020,143,5,May,21,22,Friday,23,15,4,0,23:15:00,23:15:00,05-22,23:15,Night
1638,2020-05-23 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-23,2020,144,5,May,21,23,Saturday,0,15,5,1,00:15:00,00:15:00,05-23,00:15,Late Night


In [82]:

# JMcQ - Check out missing period
plt.plot(df_5min_new[(df_5min_new['datetime']>='2020-05-20 00:00:00') & (df_5min_new['datetime']<='2020-05-30 00:00:00')]['consumption'])

In [84]:
#Get average Weekday and Weekend profiles for May and impute profile
# create_monthly_profiles(df, column)
profile_df = create_monthly_profiles(df_5min_new, 'consumption')
profile_df

timestamp  consumption       solar     alwaysOn  \
weekend monthName hour                                                       
0       April     0     1.586961e+12   152.842455   43.674318  1310.332364   
                  1     1.586965e+12   151.225545   46.842318  1310.248909   
                  2     1.586969e+12   154.196000   50.481318  1310.248909   
                  3     1.586972e+12   148.575409   46.877773  1310.248909   
                  4     1.586976e+12   151.931455   44.524773  1310.492727   
...                              ...          ...         ...          ...   
1       September 19    1.600276e+12   269.611125  138.398375  1706.781000   
                  20    1.600280e+12   195.710000   62.161000  1706.781000   
                  21    1.600284e+12   196.226000   63.231750  1706.781000   
                  22    1.600287e+12   207.688250   65.407625  1706.781000   
                  23    1.600291e+12   191.348625   61.764125  1699.147500   

                        gridImport  gridExport  selfConsumption  \
weekend monthName hour                                            
0       April     0     109.167591         0.0            100.0   
                  1     104.382727         0.0            100.0   
                  2     103.714227         0.0            100.0   
                  3     101.697182         0.0            100.0   
                  4     107.406091         0.0            100.0   
...                            ...         ...              ...   
1       September 19    131.212125         0.0            100.0   
                  20    133.548625         0.0            100.0   
                  21    132.993500         0.0            100.0   
                  22    142.280125         0.0            100.0   
                  23    129.584250         0.0            100.0   

                        selfSufficiency    year   dayOfYear  month       week  \
weekend monthName hour                                                          
0       April     0           28.739545  2020.0  106.681818    4.0  16.090909   
                  1           31.017273  2020.0  106.681818    4.0  16.090909   
                  2           32.561818  2020.0  106.681818    4.0  16.090909   
                  3           31.488636  2020.0  106.681818    4.0  16.090909   
                  4           29.461818  2020.0  106.681818    4.0  16.090909   
...                                 ...     ...         ...    ...        ...   
1       September 19          43.893750  2020.0  260.000000    9.0  37.500000   
                  20          33.783750  2020.0  260.000000    9.0  37.500000   
                  21          34.252500  2020.0  260.000000    9.0  37.500000   
                  22          33.755000  2020.0  260.000000    9.0  37.500000   
                  23          34.497500  2020.0  260.000000    9.0  37.500000   

                              day  minute  dayOfWeek  
weekend monthName hour                                
0       April     0     15.681818    15.0   2.045455  
                  1     15.681818    15.0   2.045455  
                  2     15.681818    15.0   2.045455  
                  3     15.681818    15.0   2.045455  
                  4     15.681818    15.0   2.045455  
...                           ...     ...        ...  
1       September 19    16.000000    15.0   5.500000  
                  20    16.000000    15.0   5.500000  
                  21    16.000000    15.0   5.500000  
                  22    16.000000    15.0   5.500000  
                  23    16.000000    15.0   5.500000  

[576 rows x 15 columns]

In [88]:
#weather_df = pd.read_csv("/Users/nancy/Downloads/Data/Test/data/Harare_Hourly_Weather_2005-2021.csv")
weather_df = pd.read_csv("https://raw.githubusercontent.com/OmdenaAI/NeedEnergy/Task-2-Data-Wrangling/data/weather_data/Harare_Hourly_Weather_2005-2021.csv?token=AGZP7CAUIRLTZHJM6WLHASTAHD3Z2")
weather_df.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1104537600,2005-01-01 00:00:00 +0000 UTC,7200,Harare,-17.821629,31.049226,17.07,17.85,16.06,17.40,1011,NaN,NaN,91,1.50,90,NaN,NaN,NaN,NaN,96,211,Thunderstorm,proximity thunderstorm,11n
1,1104541200,2005-01-01 01:00:00 +0000 UTC,7200,Harare,-17.821629,31.049226,16.85,18.03,16.01,17.28,1016,NaN,NaN,93,1.00,100,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
2,1104544800,2005-01-01 02:00:00 +0000 UTC,7200,Harare,-17.821629,31.049226,16.70,18.17,15.78,17.13,1017,NaN,NaN,93,0.50,90,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
3,1104548400,2005-01-01 03:00:00 +0000 UTC,7200,Harare,-17.821629,31.049226,17.23,18.84,16.70,18.43,1018,NaN,NaN,92,0.50,140,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
4,1104552000,2005-01-01 04:00:00 +0000 UTC,7200,Harare,-17.821629,31.049226,17.57,18.82,17.00,18.47,1018,NaN,NaN,88,0.82,51,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04d


#DATA QUALITY CHECK:
*Data summary:
* Raw_NeedEnergyApi_5min :
Raw_Records: 95649 with 15 columns
Cleaned(Current) :95649,

* Raw_NeedEnergyApi_hourly :
Raw_Record :7979 with 15 columns
Cleaned(Current) :7979,
* Raw_NeedEnergyApi_Daily :
Raw_Records: 335 with 15 columns
Cleaned (current):335
* Raw_NeedEnergyApi_Monthly :
Raw_records:12 with 15 columns
cleaned =12
* BASED ON THE FOLLOWING QUALITY DIMENSIONS:(ACCURACY,COMPELETENESS,VALUES FREE FROM CONTRADICTION)
THE NEEDENERGY API 5MIN,HOURLY,DAILTY,MONTHLY DATASET has zero missing values.More data required for predictive analysis,

## Feature Engineering:
More features were extracted and created from the date column feature:
* New features Generated :day,hour,month,year,day_name,
seasons(winter,autumn,spring),day sections(Night,Evening,Afternoon)

* Others:Jupyter starter files for 
Generalized Helper functions were created to generate seasons,day_sections,day,month,year,hour,quarter,were created for use for anyone willing to start-up cleaning or other task activities

* In-Progress Helper function
Public-holiday and many more for solar-iiradiance dataset.
